In [1]:
from skyfield.api import load, Topos
from skyfield.units import Angle
from skyfield import almanac
from datetime import datetime, timedelta
from pytz import timezone
from ipywidgets import widgets, interact, interactive
from IPython.display import display, HTML
import calendar

In [2]:
jkt = timezone('Asia/Jakarta')
ts = load.timescale(builtin=True)
e = load('de421.bsp')
konj = []
sunset = []

In [3]:
class var:
    loc = None

In [28]:
def find(lat, long, t0, t1):
    konj.clear()
    sunset.clear()
    topo = Topos(lat, long)
    var.loc = e['earth'] + topo
    t0 = ts.utc(t0)
    t1 = ts.utc(t1)
    
    f = almanac.oppositions_conjunctions(e, e['moon'])
    t, y = almanac.find_discrete(t0, t1, f)
    for ti, yi in zip(t, y):
        if(yi == 1):
            konj.append(t.astimezone(jkt))
            t = ti.utc
            
            t0 = ts.utc(t[0], t[1], t[2], t[3], t[4], t[5])
            t1 = ts.utc(t[0], t[1], t[2]+1, t[3], t[4], t[5])
            t, y = almanac.find_discrete(t0, t1, almanac.sunrise_sunset(e, topo))
            
            for ti, yi in zip(t, y):
                if(yi == True):
                    sunset.append(ti.astimezone(jkt))
                    bulan = getPos('moon', ti)
                    matahari = getPos('sun', ti)
                    print(bulan)
                else:
                    pass
        else:
            pass

def getPos(obj, t):
    obj = e[obj]
    astrometric = var.loc.at(t).observe(obj)
    alt, az, d = astrometric.apparent().altaz()
    return alt, az, d, astrometric

In [29]:
lat = widgets.Text(
    value='7.83305556 S',
    placeholder='Type something',
    description='Latitude:',
    disabled=False
)
long = widgets.Text(
    value='110.38305556 E',
    placeholder='Type something',
    description='Longitude:',
    disabled=False
)

hari_ini = datetime.now().date()
days_in_month = calendar.monthrange(hari_ini.year, hari_ini.month)[1]
besok = hari_ini + timedelta(days_in_month)

t0 = widgets.DatePicker(value=hari_ini,
    description='Dari:',
    disabled=False
)
t1 = widgets.DatePicker(value=besok,
    description='Sampai:',
    disabled=False
)

interact(find, lat=lat, long=long, t0=t0, t1=t1)

interactive(children=(Text(value='7.83305556 S', description='Latitude:', placeholder='Type something'), Text(…

<function __main__.find(lat, long, t0, t1)>